In [11]:
import cx_Oracle
import pandas as pd
import surprise
from surprise import Reader
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [12]:
analyzedata = pd.read_pickle("../프로젝트 1차/머신러닝/analyzedata.pkl")

In [13]:
cl0 = analyzedata[analyzedata["cluster"] == 0]
cl1 = analyzedata[analyzedata["cluster"] == 1]
cl2 = analyzedata[analyzedata["cluster"] == 2]

In [14]:
cl2_list=  list(cl2['고객번호'].iloc[:, 0])

In [15]:
db = cx_Oracle.connect('c##my', 'my', 'localhost:1521/xe')
cursor = db.cursor()
cursor.execute('select *from PURPROD4')
x = cursor.fetchall()
df = pd.DataFrame(x)
cursor.execute("select column_name from user_tab_columns where table_name = 'PURPROD4'")
col = cursor.fetchall()
list1 = []
for i in col :
    for j in i:
        list1.append(j)
df.columns = list1
df = df[df["고객번호"].isin(cl2_list)]

df.head()

,고객번호,소분류코드,임의코드,통합코드,구매빈도,구매금액,RANK0,RANK1,RANK2,RANK3
255,00002,A010403,A02,02,95,3111216,1,5,5,5.0
256,00002,A010401,A02,02,90,1802265,2,5,5,5.0
257,00002,A010402,A02,02,90,1305757,3,5,5,5.0
258,00002,A010404,A02,02,64,542568,4,5,5,5.0
259,00002,A010302,A01,01,60,419870,5,5,4,4.5


In [16]:
db = cx_Oracle.connect('c##my', 'my', 'localhost:1521/xe')
cursor = db.cursor()
cursor.execute('select *from PURPROD2')
x = cursor.fetchall()
df1 = pd.DataFrame(x)
cursor.execute("select column_name from user_tab_columns where table_name = 'PURPROD2'")
col = cursor.fetchall()
list1 = []
for i in col :
    for j in i:
        list1.append(j)
df1.columns = list1
df1.head()

,소분류코드,임의코드,소분류명,임의분류명,구매빈도
0,A010301,A01,햄,가공식품,4
1,A010302,A01,유제품,가공식품,40
2,A010304,A01,화과자,가공식품,1
3,A010505,A01,냉동식품,가공식품,4
4,A010509,A01,안주류,가공식품,1


In [17]:
reader = Reader(rating_scale = (1, 5))

data  = Dataset.load_from_df(df[['고객번호', '소분류코드', 'RANK3']], reader)

trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

algo = SVD(random_state = 0)
algo.fit(trainset)

cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1389  1.1463  1.1424  1.1438  1.1430  1.1429  0.0024  
MAE (testset)     0.9405  0.9475  0.9446  0.9441  0.9437  0.9441  0.0022  
Fit time          3.46    3.55    3.69    3.58    3.65    3.58    0.08    
Test time         17.75   1.21    1.79    1.83    1.82    4.88    6.44    


{'test_rmse': array([1.13890048, 1.14633129, 1.14240296, 1.14383879, 1.14302676]),
 'test_mae': array([0.94046389, 0.94745383, 0.94460482, 0.94410495, 0.94367008]),
 'fit_time': (3.4596259593963623,
  3.547335624694824,
  3.68833589553833,
  3.5761594772338867,
  3.646488904953003),
 'test_time': (17.749139547348022,
  1.207854986190796,
  1.7942028045654297,
  1.8330752849578857,
  1.8203041553497314)}

In [18]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs' : [20, 40, 60, 80, 100], 'n_factors' :  [50, 100, 200]}

gs = GridSearchCV(SVD, param_grid, measures = ['rmse', 'mae'], cv = 3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.1374163119782572
{'n_epochs': 20, 'n_factors': 50}


In [19]:
algo = SVD(n_epochs = 20, n_factors = 50, random_state = 0)
algo.fit(trainset)

In [20]:
def get_unpur_surprise(df, df1, 고객번호) :
    
    purs = df[df['고객번호'] == 고객번호]['소분류코드'].tolist()
    
    total_pur = df1['소분류코드'].tolist()
    
    unpurs = [pur for pur in total_pur if pur not in purs]
    
    print('평점 매긴 제품수 : ', len(purs), '추천대상 제품수 : ', len(unpurs),
         '전체 제품수 : ', len(total_pur))
    
    return unpurs

unpurs = get_unpur_surprise(df, df1, '00001')

평점 매긴 제품수 :  0 추천대상 제품수 :  6527920 전체 제품수 :  6527920


In [34]:
def recomm_pur_by_surprise(algo, 고객번호, unpurs, top_n = 5) :
    
#     preds = [algo.predict(str(고객번호), str(소분류코드)) for 소분류코드 in unpurs]
    
    preds = []
    predicted_items = set()  # 중복 호출 방지를 위한 셋(set)

    for 소분류코드 in unpurs:
        if 소분류코드 not in predicted_items:  # 이미 예측한 소분류코드인지 확인
            pred = algo.predict(str(고객번호), str(소분류코드))  # 예측 객체 생성
            preds.append(pred)  # 예측 객체를 리스트에 추가
            predicted_items.add(소분류코드)  # 예측한 소분류코드를 셋에 추가
    
    def sortkey_est(pred) :
        
        return pred.est
    
    preds.sort(key = sortkey_est, reverse = True)
    top_predictions = preds[:top_n]
    
    top_pur_ids = [str(pred.iid) for pred in top_predictions]
    top_pur_rating = [pred.est for pred in top_predictions]
    top_pur_titles = df1[df1.소분류코드.isin(top_pur_ids)]['소분류명']
    top_pur_preds = [(id, title, rating) for id, title, rating in zip(top_pur_ids, top_pur_titles, top_pur_rating)]
    
    return top_pur_preds

unpurs = get_unpur_surprise(df, df, '00073')
top_pur_preds = recomm_pur_by_surprise(algo, '00073', unpurs, top_n = 5)
print('#### Top-5 추천 제품 리스트 ####')

for top_pur in top_pur_preds:
    print(top_pur[1], ':', top_pur[2])

평점 매긴 제품수 :  157 추천대상 제품수 :  370570 전체 제품수 :  450628
#### Top-5 추천 제품 리스트 ####
수입 아동복 : 4.27528009104
L/C 아웃도어 : 4.148186812501966
수입화 : 4.138557403224032
수입화 : 4.09581737798784
명품잡화 : 4.026836159852552


In [26]:
df.RANK1.unique()

array([5, 4, 3, 2, 1], dtype=int64)

In [27]:
cl2.sort_values("구매 총액", ascending=False)["고객번호"][:50]

,고객번호,고객번호
8078,08079,08079
96,00097,00097
11762,11763,11763
2289,02290,02290
946,00947,00947
9517,09518,09518
561,00562,00562
9405,09406,09406
8101,08102,08102
9775,09776,09776


In [5]:
# df.to_pickle('purprod4.pkl')

In [6]:
# p = pd.read_pickle('purprod4.pkl')

# p.head()

,고객번호,소분류코드,임의코드,통합코드,구매빈도,RANK0,RANK1
0,00001,A010401,A02,02,60,1,5
1,00001,A010404,A02,02,59,2,5
2,00001,A010302,A01,01,40,3,5
3,00001,A040222,A09,09,29,4,5
4,00001,A020302,A03,03,28,5,5
